In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#playlists_df = pd.read_csv('playlists_new.csv')
tracks_df = pd.read_csv('tracks_new.csv', index_col='Unnamed: 0')
#users_df = pd.read_csv('users_new.csv')

In [3]:
tracks_df.shape

(6479, 37)

Giả sử, một người nghe lại bài hát đó chỉ khi người đó đã like
Nên: số lượt like phải nhỏ hơn lượt nghe lại. 
Kiểm tra giả thuyết này, nếu số dòng đủ nhỏ ta có thể remove

In [4]:
tracks_df[tracks_df['likes_count'] > tracks_df['playback_count']].shape

(4, 37)

Tiến hành loại bỏ các dòng này.

In [5]:
tracks_df = tracks_df[tracks_df['likes_count'] < tracks_df['playback_count']]
tracks_df.shape

(6279, 37)

Loại bỏ các dòng có title trùng, giữ dòng đầu tiên

In [6]:
tracks_df.drop_duplicates(subset=['title'], keep='first', inplace=True)
tracks_df.shape

(5349, 37)

Tìm tên của 20 tracks được like nhiều nhất và xếp thứ tự giảm dần likes_count

In [7]:
top_likes = tracks_df.sort_values(by='likes_count', ascending=False)
top_likes[['title','likes_count']].head(20)

,title,likes_count
Unnamed: 0,,
1364,Major Lazer & DJ Snake - Lean On (feat. MØ),3055218.0
2959,"Desiigner- ""Panda"" (Prod. By: Menace)",2904933.0
2918,Remy Boyz - My Way RMX Ft. Drake,1507429.0
2856,Martin Garrix & Bebe Rexha - In The Name Of Love,1386074.0
2913,Gucci Mane - Both (Ft. Drake),1191522.0
2896,Alone (Original Mix),1047919.0
2899,Major Lazer - Light It Up (Feat. NYLA & Fuse O...,969743.0
243,Sunset Lover,882370.0
3008,Skrillex & Rick Ross - Purple Lamborghini,702231.0


Nhập vào danh sách tên các nghệ sĩ, tìm tổng số lượt like của các nghệ sĩ này:
- Giả sử tên của nghệ sĩ phải nằm trong 'title'
- Đối với các dòng duplicate 'title', không remove duplicate mà vẫn giữ nguyên để xử lí
- Có thể có nhiều nghệ sĩ tham gia trong cùng 1 title
- Kết quả trả ra sắp xếp giảm dần số lượt like

In [8]:
def top_artists(artists):
    artists_df = pd.DataFrame(columns=['sum_likes_count'], index = artists)
    for artist in artists:
        artists_df.loc[artist, 'sum_likes_count'] = 0
    
    for i in tracks_df.index:
        for artist in artists:
            a = tracks_df.loc[i, 'title'].find(artist)
            if a >= 0:#found
                artists_df.loc[artist,'sum_likes_count']=artists_df.loc[artist,'sum_likes_count']+tracks_df.loc[i, 'likes_count']
        
    artists_df.sort_values(by='sum_likes_count',ascending=False, inplace=True)
    return artists_df     

In [9]:
artists = ['Snake','Martin Garrix','Chainsmokers','Alan Walker']
top_artists_df = top_artists(artists)
top_artists_df 

,sum_likes_count
Snake,3420625.0
Martin Garrix,1541333.0
Alan Walker,1290670.0
Chainsmokers,945538.0


Mục đích: tìm hiểu số lượt like của thần tượng để cày view

Người dùng thường dựa vào likes_count hoặc playback_count để đánh giá sự phổ biến của một bài hát. Kiểm tra trong top 'x' likes_count nhiều nhất với top 'x' playback_count và sự chênh lệch là bao nhiêu. Với x là top 10, 100, 1000

In [10]:
def top_ratio(first:int):
    top_likes = tracks_df.sort_values(by='likes_count', ascending=False).head(first)
    top_pb = tracks_df.sort_values(by='playback_count', ascending=False).head(first)
    top_likes = top_likes.append(top_pb)
    dup = top_likes.duplicated(keep='first').value_counts()[True]
    return dup/first

In [11]:
print('Tỉ lệ trùng nhau của top 10: ', top_ratio(10))
print('Tỉ lệ trùng nhau của top 100: ', top_ratio(100))
print('Tỉ lệ trùng nhau của top 1000: ', top_ratio(1000))

Tỉ lệ trùng nhau của top 10:  0.8
Tỉ lệ trùng nhau của top 100:  0.88
Tỉ lệ trùng nhau của top 1000:  0.85


Kết luận: nếu một bài hát nằm trong top những bài hát có lượt likes_counts cao nhất, không có nghĩa là nó cũng sẽ nằm trong top lượt playback_counts cao nhất